<a href="https://colab.research.google.com/github/tsakailab/spmlib/blob/master/demo/eg01_GreedyAlgorithms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline

In [ ]:
# S .G. Mallat and Z. Zhang. "Matching pursuits with time-frequency dictionaries."
# IEEE TSP 41(12), pp. 3397-3415, 1993.
# Matching Pursuit (MP)
import numpy as np
from scipy.linalg import norm
def MP(A, b, delta=1e-6, maxnnz=None):
    m, n = A.shape
    if maxnnz is None: maxnnz = m//2
    scale = 1.0 / norm(A, axis=0)
    r = b.copy()
    x = np.zeros(n, dtype=A.dtype)
    supp = set()
    while len(supp) < maxnnz and norm(r) > delta:
        c = A.conj().T.dot(r) * scale
        s = np.argmax(np.abs(c))
        supp.add(s)
        dx = c[s] * scale[s]
        x[s] += dx
        r -= A[:,s] * dx
    return x

In [ ]:
# Y.C. Pati, R. Rezaiifar, Y.C.P.R. Rezaiifar, and P.S. Krishnaprasad,
# "Orthogonal matching pursuit: Recursive function approximation with applications to wavelet decomposition,"
# Proc. 27th Annual Asilomar Conference on Signals, Systems, and Computers, pp.40–44, 1993.
# Orthogonal matching pursuit (OMP)
import numpy as np
from scipy.linalg import norm, lstsq
def OMP(A, b, delta=1e-6, maxnnz=None):
    m, n = A.shape
    if maxnnz is None: maxnnz = m // 2
    r = b.copy()
    x = np.zeros(n, dtype=b.dtype)
    supp = []
    while len(supp) < maxnnz and norm(r) > delta:
        s = np.argmax(np.abs(A.T.dot(r)))
        supp.append(s)
        Asupp = A[:,supp]
        x[supp] = lstsq(Asupp, b)[0]
        r = b - Asupp.dot(x[supp])
    x[np.abs(x) < np.spacing(np.single(np.abs(x).max()))] = 0.
    return x

In [ ]:
# J. Wang, S. Kwon, and B. Shim, "Generalized orthogonal matching pursuit,"
# IEEE TSP 60(12), pp.6202–6216, 2012.
# Generalized orthogonal matching pursuit (gOMP)
import numpy as np
from scipy.linalg import norm, lstsq
def gOMP(A, b, N=3, delta=1e-6, maxiter=None):
    m, n = A.shape
    if maxiter is None: maxiter = m // N
    r = b.copy()
    x = np.zeros(n, dtype=b.dtype)
    supp = []

    count = 0
    while count < maxiter and norm(r) > delta:
        count += 1
        s = np.argsort(-np.abs(A.T.dot(r)))
        supp.extend(s[:N])
        Asupp = A[:,supp]
        x[supp] = lstsq(Asupp, b)[0]
        r = b - Asupp.dot(x[supp])
    x[np.abs(x) < np.spacing(np.single(np.abs(x).max()))] = 0.
    return x

In [ ]:
import numpy as np
from scipy.linalg import norm, qr, qr_insert, solve_triangular

# Generalized orthogonal matching pursuit with QR factorization (gOMPQR)
def gOMPQR(A, b, N=3, delta=1e-6, maxiter=None):
    m, n = A.shape
    if maxiter is None: maxiter = m // N
    r = b.copy()

    # find the 1st atom group and orthonormal basis
    s = np.argsort(-np.abs(A.T.dot(r)))
    Q, R = qr(A[:,s[:N]], mode='economic')
    xi = Q.T.dot(b)
    r -= Q.dot(xi)
    xnz = xi.tolist()
    supp = s[:N].tolist()
    
    count = 1
    while count < maxiter and norm(r) > delta:
        count += 1
        s = np.argsort(-np.abs(A.T.dot(r)))
        Q, R = qr_insert(Q, R, A[:,s[:N]], Q.shape[1], which='col')
        xi = Q[:,-N:].T.dot(b)
        r -= Q[:,-N:].dot(xi)
        xnz.extend(xi)
        supp.extend(s[:N])

    xnz = solve_triangular(R, xnz)
    xnz[np.abs(xnz) < np.spacing(np.single(np.abs(xnz).max()))] = 0.
    x = np.zeros(n, dtype=xnz.dtype)
    x[supp] = xnz
    return x

In [ ]:
from time import time
# Generate sythetic data for demo
rng = np.random.RandomState(int(time()))
#m, n = 128, 256
#m, n = 256, 512
#m, n = 512, 2048
#m, n = 1024, 8192
m, n = 2000, 4000
dtype = np.float64  # try np.float32 and np.float64

# use a random matrix as a basis
A = rng.randn(m, n).astype(dtype) / np.sqrt(m)

# generate a k-sparse Gaussian signal vector
k = 300
stdx = 1.
snr = 10.  # try 20., 10., 5., np.inf

x_true = np.zeros(n, dtype=dtype)
T = np.sort(rng.choice(n,k,replace=False))
print('True support of %d/%d nonzeros = ' % (k,n))
print(T)
x_true[T] = rng.randn(k).astype(dtype) * stdx

# make the query vector
b = A.dot(x_true)

# add noise
if np.isinf(snr):
    noise = 0.
else:
    normb = norm(b)
    noise = rng.randn(m).astype(dtype)
    noise = noise / norm(noise) * normb / snr
    b = b + noise

tol = norm(noise)

In [ ]:
# OMP
print("Running OMP..")
t0 = time()
x_est = OMP(A, b, delta=tol)
#x_est = OMP(A, b)  # noiseless case (snr=np.inf)
print('done in %.2fs.' % (time() - t0))

Tpred = np.nonzero(x_est)[0]
print('Predicted supprt of %d nonzeros = ' % (np.count_nonzero(x_est)))
print(Tpred)

from sklearn import metrics
print(metrics.classification_report(x_true == 0, x_est == 0))
print(metrics.confusion_matrix(x_true == 0, x_est == 0))

In [ ]:
# gOMP
print("Running gOMP..")
t0 = time()
x_est = gOMP(A, b, N=6, delta=tol)
#x_est = gOMP(A, b, N=6)  # noiseless case (snr=np.inf)
print('done in %.2fs.' % (time() - t0))

Tpred = np.nonzero(x_est)[0]
print('Predicted supprt of %d nonzeros = ' % (np.count_nonzero(x_est)))
print(Tpred)

from sklearn import metrics
print(metrics.classification_report(x_true == 0, x_est == 0))
print(metrics.confusion_matrix(x_true == 0, x_est == 0))

In [ ]:
# gOMPQR
print("Running gOMPQR..")
t0 = time()
x_est = gOMPQR(A, b, N=6, delta=tol)
#x_est = gOMPQR(A, b, N=6, maxiter=min(k,m//6))
#x_est = gOMPQR(A, b, N=6)  # noiseless case (snr=np.inf)
print('done in %.2fs.' % (time() - t0))

Tpred = np.nonzero(x_est)[0]
print('Predicted supprt of %d nonzeros = ' % (np.count_nonzero(x_est)))
print(Tpred)

from sklearn import metrics
print(metrics.classification_report(x_true == 0, x_est == 0))
print(metrics.confusion_matrix(x_true == 0, x_est == 0))

In [ ]:
import matplotlib.pyplot as plt
from scipy.linalg import norm

def plot_sparse_solution(x_true, x_est, figname=None):
    xrelerr = norm(x_est-x_true)/norm(x_true)
    print('rel. error = %.2e' % (xrelerr))
    
    x_true_nz = x_true.nonzero()[0]
    plt.plot(x_true_nz, x_true[x_true_nz], 
             '.', mfc='k', markersize=6, mec='k', label='True (nnz='+str(len(x_true_nz))+')')
    x_est_nz = x_est.nonzero()[0]
    plt.plot(x_est_nz, x_est[x_est_nz], 
             'o', mfc = 'None', markersize=8, mec='0.5', mew=1, label='Estimated (nnz='+str(len(x_est_nz))+')')
    plt.xlim(0, len(x_true))
    plt.axhline(0, color='k', linewidth=1)

    plt.legend(loc='upper right', shadow=False)
    plt.legend(fontsize=12)
    plt.tick_params(labelsize=16)

    plt.tight_layout()
    if figname is not None:
        plt.savefig(figname+'.pdf', bbox_inches='tight', dpi=300)
        plt.savefig(figname+'.eps', bbox_inches='tight', dpi=300)

In [ ]:
plot_sparse_solution(x_true, x_est)#, figname='gOMP_solution')

In [ ]:
%timeit -r 4 -n 10 gOMPQR(A, b, N=6,delta=tol)

In [ ]:
#m, n = 512, 1024
m, n = 128, 256
reconsts = {'test_count': 0, 'shape': (m,n), 'nnz': range(5, (3*m)//4, 10)}
for alg in ['OMP', 'gOMP3', 'gOMP6', 'gOMP9']:
    reconsts[alg] = {'success': {}, 'time': {}}
    for k in reconsts['nnz']:
        reconsts[alg]['success'][k] = 0
        reconsts[alg]['time'][k] = 0
print(reconsts)

In [ ]:
from sklearn import metrics

print(reconsts['nnz'])
stdx = 1.
snr = np.inf  # try 20., 10., 5.
rng = np.random.RandomState(int(time()))
dtype = np.float64  # try np.float32 and np.float64

def count_success(x_true, x_est, reconsts, alg, k):
    cf = metrics.confusion_matrix(x_true == 0, x_est == 0)
    if cf[1,0] == 0 and cf[0,1] == 0:
        reconsts[alg]['success'][k] += 1

ntests = 100
reconsts['test_count'] += ntests
for k in reconsts['nnz']:

    x_true = np.zeros(n, dtype=dtype)
    T = np.sort(rng.choice(n,k,replace=False))
    # print('True support of %d nonzeros = ' % (k))
    x_true[T] = rng.randn(k).astype(dtype) * stdx

    for t in range(ntests):
        # use a random matrix as a basis
        A = rng.randn(m, n).astype(dtype) / np.sqrt(m)

        # make the query vector
        b = A.dot(x_true)

        # add noise
        if np.isinf(snr):
            noise = 0.
        else:
            normb = norm(b)
            noise = rng.randn(m).astype(dtype)
            noise = noise / norm(noise) * normb / snr
            b = b + noise
    
        tol = norm(noise)
        
        t0 = time()   
        x_est = gOMPQR(A, b, N=1, maxiter=min(k, m//2), delta=norm(b)*1e-8)
        reconsts['OMP']['time'][k] += time() - t0
        count_success(x_true, x_est, reconsts, 'OMP', k)

        t0 = time()   
        x_est = gOMPQR(A, b, N=3, maxiter=min(k, m//3), delta=norm(b)*1e-8)
        reconsts['gOMP3']['time'][k] += time() - t0
        count_success(x_true, x_est, reconsts, 'gOMP3', k)

        t0 = time()   
        x_est = gOMPQR(A, b, N=6, maxiter=min(k, m//6), delta=norm(b)*1e-8)
        reconsts['gOMP6']['time'][k] += time() - t0
        count_success(x_true, x_est, reconsts, 'gOMP6', k)

        t0 = time()   
        x_est = gOMPQR(A, b, N=9, maxiter=min(k, m//9), delta=norm(b)*1e-8)
        reconsts['gOMP9']['time'][k] += time() - t0
        count_success(x_true, x_est, reconsts, 'gOMP9', k)


In [ ]:
import matplotlib.pyplot as plt
plt.figure()

reconst = np.array(list(reconsts['OMP']['success'].values())) / reconsts['test_count']
plt.plot(reconsts['nnz'], reconst, 'ko-.', markersize=6, mec='k', mfc='None', label='OMP')

reconst = np.array(list(reconsts['gOMP3']['success'].values())) / reconsts['test_count']
plt.plot(reconsts['nnz'], reconst, 'ks-', markersize=6, mec='k', mfc='None', label='gOMP (N=3)')

reconst = np.array(list(reconsts['gOMP6']['success'].values())) / reconsts['test_count']
plt.plot(reconsts['nnz'], reconst, 'k*-', markersize=8, mec='k', mfc='None', label='gOMP (N=6)')

reconst = np.array(list(reconsts['gOMP9']['success'].values())) / reconsts['test_count']
plt.plot(reconsts['nnz'], reconst, 'k>-', markersize=6, mec='k', mfc='None', label='gOMP (N=9)')

#plt.plot(np.arange(n), x_est, 'ro', mfc = 'None', markersize=8, mec='red', mew=1, label='Estimated')
plt.legend(loc='lower left', shadow=False)
plt.title('#trials='+str(reconsts['test_count']), fontsize=16)
plt.xlabel("Sparsity", fontsize=16)
plt.ylabel("Prob. of exact reconst.", fontsize=16)
plt.legend(fontsize=16)
plt.tick_params(labelsize=16)

plt.tight_layout()
#plt.savefig('gOMP_prob_reconst.pdf', bbox_inches='tight', dpi=300)
#plt.savefig('gOMP_prob_reconst.eps', bbox_inches='tight', dpi=300)

plt.show()

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
#plt.yscale("log")
from matplotlib.ticker import ScalarFormatter

reconst = np.array(list(reconsts['OMP']['time'].values())) / reconsts['test_count']
plt.plot(reconsts['nnz'], reconst*1000, 'ko-.', markersize=6, mec='k', mfc='None', label='OMP')

reconst = np.array(list(reconsts['gOMP3']['time'].values())) / reconsts['test_count']
plt.plot(reconsts['nnz'], reconst*1000, 'ks-', markersize=6, mec='k', mfc='None', label='gOMP (N=3)')

reconst = np.array(list(reconsts['gOMP6']['time'].values())) / reconsts['test_count']
plt.plot(reconsts['nnz'], reconst*1000, 'k*-', markersize=8, mec='k', mfc='None', label='gOMP (N=6)')

reconst = np.array(list(reconsts['gOMP9']['time'].values())) / reconsts['test_count']
plt.plot(reconsts['nnz'], reconst*1000, 'k>-', markersize=6, mec='k', mfc='None', label='gOMP (N=9)')

#plt.plot(np.arange(n), x_est, 'ro', mfc = 'None', markersize=8, mec='red', mew=1, label='Estimated')
plt.legend(loc='upper left', shadow=False)
plt.title('#trials='+str(reconsts['test_count']), fontsize=16)
plt.xlabel("Sparsity", fontsize=16)
plt.ylabel("Running time [ms]", fontsize=16)
plt.legend(fontsize=16)
plt.tick_params(labelsize=16)

plt.tight_layout()
#plt.savefig('gOMP_runningtime.pdf', bbox_inches='tight', dpi=300)
#plt.savefig('gOMP_runningtime.eps', bbox_inches='tight', dpi=300)
plt.show()

In [ ]:
# SPMLIB https://github.com/tsakailab/spmlib
%%capture
!git clone https://github.com/tsakailab/spmlib.git
%cd spmlib
!python setup.py develop --user

from spmlib import solver as sps

In [ ]:
from numpy import linalg
# MP
print("Running MP..")
t0 = time()
result_MP = sps.matching_pursuit(A, b, tol=tol)
#result_MP = sps.matching_pursuit_LinearOperator(splinalg.aslinearoperator(A), b, tol=tol)
x_est = result_MP[0]
print('done in %.2fs.' % (time() - t0))
print('%d iterations, supprt of %d nonzeros = ' % (result_MP[2], np.count_nonzero(x_est)))
print(np.nonzero(x_est)[0])
print('rel. error = %.2e' % (linalg.norm(x_est-x_true)/linalg.norm(x_true)))

plt.figure()
#plt.stem(x_true, markerfmt='g.', label='True')
plt.plot(np.arange(n), x_true, 'g.', markersize=8, mec='green', label='True')
plt.plot(np.arange(n), x_est, 'ro', mfc = 'None', markersize=8, mec='red', label='Estimated')
plt.legend(loc='upper right', shadow=False)
plt.show()



# OMP
print("Running OMP..")
t0 = time()
result_OMP = sps.orthogonal_matching_pursuit(A, b, tol=tol)
#result_OMP = sps.orthogonal_matching_pursuit_using_linearoperator(splinalg.aslinearoperator(A), b, tol=tol)
#result_OMP = OMP(A, b, tol=linalg.norm(b)*1e-4, max_nnz=100)
x_est = result_OMP[0]
print('done in %.2fs.' % (time() - t0))
print('%d iterations, supprt of %d nonzeros = ' % (result_OMP[2], np.count_nonzero(x_est)))
print(np.nonzero(x_est)[0])
print('rel. error = %.2e' % (linalg.norm(x_est-x_true)/linalg.norm(x_true)))

plt.figure()
#plt.stem(x_true, markerfmt='g.', label='True')
plt.plot(np.arange(n), x_true, 'g.', markersize=8, mec='green', label='True')
plt.plot(np.arange(n), x_est, 'ro', mfc = 'None', markersize=8, mec='red', label='Estimated')
plt.legend(loc='upper right', shadow=False)
plt.show()


# Generalized OMP
print("Running gOMP..")
t0 = time()
result_gOMP = sps.generalized_orthogonal_matching_pursuit(A, b, N=5, tol=tol)
x_est = result_gOMP[0]
print('done in %.2fs.' % (time() - t0))
print('%d iterations, supprt of %d nonzeros = ' % (result_gOMP[2], np.count_nonzero(x_est)))
print(np.nonzero(x_est)[0])
print('rel. error = %.2e' % (linalg.norm(x_est-x_true)/linalg.norm(x_true)))

plt.figure()
#plt.stem(x_true, markerfmt='g.', label='True')
plt.plot(np.arange(n), x_true, 'g.', markersize=8, mec='green', label='True')
plt.plot(np.arange(n), x_est, 'ro', mfc = 'None', markersize=8, mec='red', label='Estimated')
plt.legend(loc='upper right', shadow=False)
plt.show()


# subspace pursuit
print("Running subspace pursuit..")
t0 = time()
result_SP = sps.subspace_pursuit(A, b)
x_est = result_SP[0]
print('done in %.2fs.' % (time() - t0))
print('%d iterations, supprt of %d nonzeros = ' % (result_SP[2], np.count_nonzero(x_est)))
print(np.nonzero(x_est)[0])
print('rel. error = %.2e' % (linalg.norm(x_est-x_true)/linalg.norm(x_true)))

plt.figure()
#plt.stem(x_true, markerfmt='g.', label='True')
plt.plot(np.arange(n), x_true, 'g.', markersize=8, mec='green', label='True')
plt.plot(np.arange(n), x_est, 'ro', mfc = 'None', markersize=8, mec='red', label='Estimated')
plt.legend(loc='upper right', shadow=False)
plt.show()


# FISTA followed by LS debias
#l = 0.1*stdx
l = (stdx / k * m) / np.sqrt(snr) / normb
print("Running FISTA followed by debias..")
t0 = time()
result_FISTA_debias = sps.fista(A, b, tol=tol, l=l, tolx=linalg.norm(A.T.dot(b))*1e-5, maxiter=50, debias=True)
x_est = result_FISTA_debias[0]
print('done in %.2fs.' % (time() - t0))
print('%d iterations, supprt of %d nonzeros = ' % (result_FISTA_debias[2], np.count_nonzero(x_est)))
print(np.nonzero(x_est)[0])
print('rel. error = %.2e' % (linalg.norm(x_est-x_true)/linalg.norm(x_true)))

plt.figure()
#plt.stem(x_true, markerfmt='g.')
plt.plot(np.arange(n), x_true, 'g.', markersize=8, mec='green', label='True')
plt.plot(np.arange(n), x_est, 'ro', mfc = 'None', markersize=8, mec='red', label='Estimated')
plt.legend(loc='upper right', shadow=False)
plt.show()